# FashionMNIST Image Classification with ResNet18 fine-tuning

## Overview
This notebook implements a Transfer Learning approach to classify images from the **FashionMNIST** dataset. We adapt a pre-trained **ResNet18** architecture (originally designed for ImageNet) to handle 28x28 grayscale fashion items.

## Technical Approach
* **Model:** ResNet18 (Residual Neural Network) with modified input/output layers.
* **Regularization:** Data Augmentation (Rotation, Flipping) and Dropout.
* **Optimization:** Adam Optimizer with CrossEntropyLoss.

In [ ]:
#!pip install datasets
#!pip install torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , random_split
from torchvision import datasets , models , transforms 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(42)
np.random.seed(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Augmentation & Loading
Utilizing `torchvision.transforms` to "expand" the training dataset and prevent overfitting.
* **Resize:** Upscaling images to 224x224 (standard input for ResNet).
* **Random Rotation (15°):** Makes the model robust to orientation changes.
* **Random Horizontal Flip:** Simulates different viewing angles.

In [ ]:
batch_size = 64
learning_rate = 1e-4
epochs = 10
hidden_units = 256
resize_dim = 224

train_transform = transforms.Compose( [ transforms.Resize((resize_dim , resize_dim)) , transforms.RandomHorizontalFlip(p=0.5) ,
                                       transforms.RandomRotation(degrees=15) , transforms.ToTensor() , transforms.Normalize(mean=[0.5] , std = [0.5])
                                      ])
val_test_transform = transforms.Compose( [ transforms.Resize((resize_dim , resize_dim)) , transforms.ToTensor() , transforms.Normalize(mean = [0.5] , std = [0.5]) ])

full_train_ds = datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transform)
test_ds = datasets.FashionMNIST(root='./data', train=False, download=True, transform=val_test_transform)

train_size = int(0.8*len(full_train_ds))
dev_size = len(full_train_ds) - train_size
train_ds , dev_ds = random_split(full_train_ds , [train_size , dev_size])

train_loader = DataLoader(train_ds , batch_size = batch_size , shuffle = True , num_workers = 2)
dev_loader = DataLoader(dev_ds , batch_size = batch_size , shuffle = False , num_workers = 2)
test_loader = DataLoader(test_ds , batch_size = batch_size , shuffle = False , num_workers = 2)
final_train_loader = DataLoader(full_train_ds , batch_size = batch_size , shuffle = True , num_workers = 2)

## Model Architecture: Adapted ResNet18
The standard ResNet18 expects 3-channel RGB images (ImageNet). We modify it to work with FashionMNIST (1-channel):

1.  **Input Layer:** Changed the first Convolutional layer (`conv1`) to accept **1 channel** (grayscale) instead of 3.
2.  **Backbone:** Using the pre-trained weights (`ResNet18_Weights.DEFAULT`) to leverage learned feature extractors.
3.  **Classifier Head:** Replacing the final Fully Connected layer with a custom MLP:
    * Linear Layer -> ReLU -> Dropout (0.5) -> Output Layer (10 classes).

In [ ]:
class FashionMNISTmodel(nn.Module):
    def __init__(self , hidden_units , num_classes=10):
        super(FashionMNISTmodel , self).__init__()
        self.resnet = models.resnet18( weights = models.ResNet18_Weights.DEFAULT)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        in_features = self.resnet.fc.in_features

        self.resnet.fc = nn.Sequential( nn.Linear(in_features , hidden_units) , nn.ReLU() , nn.Dropout(0.5) , nn.Linear(hidden_units , num_classes))

    def forward(self , x):
            return self.resnet(x)

model = FashionMNISTmodel(hidden_units = hidden_units).to(device)
optimizer = optim.Adam(model.parameters() , lr = learning_rate)
criterion = nn.CrossEntropyLoss()

## Training & Validation
Training the model for 10 epochs using the **Adam** optimizer . The loss is tracked on both Training and Development sets to identify the optimal stopping point (best epoch).

In [ ]:
train_losses = []
dev_losses = []
#εύρεση καλύτερου αριθμού εποχών
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for images , labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs , labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    average_train_loss = train_loss / len(train_loader)
    train_losses.append(average_train_loss)

    model.eval()
    dev_loss = 0.0
    with torch.no_grad():
        for images , labels in dev_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs , labels)
            dev_loss += loss.item()

    average_dev_loss = dev_loss / len(dev_loader)
    dev_losses.append(average_dev_loss)

In [ ]:
best_epoch = np.argmin(dev_losses) + 1

epochs_range = range(1, len(train_losses) + 1)
plt.figure(figsize=(10, 5))
plt.plot(epochs_range ,train_losses, label='Train Data Loss ')
plt.plot(epochs_range , dev_losses, label='Dev Data Loss ')
plt.title('Loss Per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(epochs_range)
plt.legend()
plt.grid(True)
plt.show()

## Final Evaluation
Identifying the best epoch, retraining the model on the **Full Training Set** and evaluating it on the **Test Set**.

The table below details the Precision, Recall, and F1-Score for each of the 10 fashion categories.

In [ ]:
final_model = FashionMNISTmodel(hidden_units = hidden_units).to(device)
final_optimizer = optim.Adam(final_model.parameters() , lr = learning_rate)

final_model.train()
for epoch in range(best_epoch):
    final_train_loss = 0.0
    for images , labels in final_train_loader:
        images = images.to(device)
        labels = labels.to(device)
        final_optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs , labels)
        loss.backward()
        final_optimizer.step()
        final_train_loss += loss.item()

    average_loss = final_train_loss / len(final_train_loader)

all_preds = []
all_labels = []
final_model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = final_model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

report_dict = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()

print("Test Data Stats")
print(df_report[['precision', 'recall', 'f1-score']])